In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and preprocess dataset
file_path = "creditcard.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')

# Drop rows with missing target labels
data = data.dropna(subset=['Class'])

# Separate features and labels
X = data.drop(columns=['Class'])
y = data['Class']

# Normalize features (excluding Time)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.drop(columns=['Time']))
X_scaled = pd.DataFrame(X_scaled, columns=X.columns[1:])
X_scaled['Time'] = X['Time']  # Add Time back unscaled

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Reshape target arrays to 2D
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

# Create federated clients
num_clients = 3
client_data = []
split_X = np.array_split(X_train, num_clients)
split_y = np.array_split(y_train, num_clients)
for i in range(num_clients):
    client_data.append({'X': split_X[i].values, 'y': split_y[i]})

# Define test data
test_data = {'X': X_test.values, 'y': y_test}

# Define model
def create_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Federated SGD function
def train_fedsgd(global_model, client_datasets, test_data, num_rounds, learning_rate):
    global_weights = global_model.get_weights()

    for round_num in range(num_rounds):
        print(f"Starting round {round_num + 1}/{num_rounds}...")
        gradients_list = []

        # Collect gradients from clients
        for client_idx, client_data in enumerate(client_datasets):
            with tf.GradientTape() as tape:
                # Clone the global model
                local_model = create_model(global_model.input_shape[-1])
                local_model.set_weights(global_weights)

                # Predict and calculate loss
                predictions = local_model(client_data['X'], training=True)
                loss = tf.keras.losses.binary_crossentropy(client_data['y'], predictions)

            # Compute gradients
            gradients = tape.gradient(loss, local_model.trainable_weights)
            gradients_list.append(gradients)
            print(f"Client {client_idx + 1} loss: {tf.reduce_mean(loss).numpy():.4f}")

        # Average gradients across clients
        averaged_gradients = [tf.reduce_mean([g[layer] for g in gradients_list], axis=0)
                              for layer in range(len(global_weights))]

        # Update global weights using averaged gradients
        global_weights = [
            w - learning_rate * g for w, g in zip(global_weights, averaged_gradients)
        ]
        global_model.set_weights(global_weights)

        # Evaluate global model on test data
        global_loss, global_acc = global_model.evaluate(test_data['X'], test_data['y'], verbose=0)
        print(f"Global Model Accuracy: {global_acc:.4f} (after round {round_num + 1})")

    return global_model

# Initialize global model
input_dim = X_train.shape[1]
global_model = create_model(input_dim)

# Compile global model (required for evaluation)
global_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

# Federated training with SGD
num_rounds = 5
learning_rate = 0.01
global_model = train_fedsgd(global_model, client_data, test_data, num_rounds, learning_rate)
print("Federated training completed.")


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Starting round 1/5...
Client 1 loss: 4.3134
Client 2 loss: 2.6673
Client 3 loss: 2.9781
Global Model Accuracy: 0.0042 (after round 1)
Starting round 2/5...
Client 1 loss: 52533018624.0000
Client 2 loss: 52841328640.0000
Client 3 loss: 52467429376.0000
Global Model Accuracy: 0.9958 (after round 2)
Starting round 3/5...
Client 1 loss: 23601983488.0000
Client 2 loss: 14592890880.0000
Client 3 loss: 16288781312.0000
Global Model Accuracy: 0.0042 (after round 3)
Starting round 4/5...
Client 1 loss: 2628849821399990476472320.0000
Client 2 loss: 2644277928744263150993408.0000
Client 3 loss: 2625567453876374783131648.0000
Global Model Accuracy: 0.9958 (after round 4)
Starting round 5/5...
Client 1 loss: 1262274110846471766016.0000
Client 2 loss: 965268470820828610560.0000
Client 3 loss: 891296565466291503104.0000
Global Model Accuracy: 0.9954 (after round 5)
Federated training completed.


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and preprocess dataset
file_path = "creditcard.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')

# Handle missing values
data.dropna(inplace=True)

# Separate features and labels
X = data.drop(columns=['Class'])
y = data['Class']

# Normalize features (excluding Time)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.drop(columns=['Time']))
X_scaled = pd.DataFrame(X_scaled, columns=X.columns[1:])
X_scaled['Time'] = X['Time']  # Add Time back unscaled

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Create federated clients
num_clients = 3
client_data = []
split_X = np.array_split(X_train, num_clients)
split_y = np.array_split(y_train, num_clients)
for i in range(num_clients):
    client_data.append({'X': split_X[i].values, 'y': split_y[i].values})

# Define test data
test_data = {'X': X_test.values, 'y': y_test.values}

# Define model
def create_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Federated SGD training
def train_fedsgd(global_model, client_datasets, test_data, num_rounds, learning_rate):
    global_weights = global_model.get_weights()

    for round_num in range(num_rounds):
        print(f"Starting round {round_num + 1}/{num_rounds}...")
        client_losses = []

        # Local training on each client
        for client_idx, client_data in enumerate(client_datasets):
            # Clone the global model for local training
            local_model = create_model(global_model.input_shape[-1])
            local_model.set_weights(global_weights)

            # Compile the local model with gradient clipping
            optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, clipnorm=1.0)
            local_model.compile(optimizer=optimizer, loss='binary_crossentropy')

            # Train locally for one batch
            history = local_model.fit(client_data['X'], client_data['y'], epochs=1, verbose=0, batch_size=32)

            # Calculate client loss
            local_loss = history.history['loss'][-1]
            client_losses.append(local_loss)
            print(f"Client {client_idx + 1} loss: {local_loss:.4f}")

            # Update global weights (FedSGD: average updates)
            global_weights = [
                global_weight + learning_rate * (local_weight - global_weight)
                for global_weight, local_weight in zip(global_weights, local_model.get_weights())
            ]

        # Set updated global weights
        global_model.set_weights(global_weights)

        # Evaluate global model on test dataset
        global_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        global_loss, global_acc = global_model.evaluate(test_data['X'], test_data['y'], verbose=0)
        print(f"Global Model Accuracy: {global_acc:.4f} (after round {round_num + 1})")

    return global_model

# Training parameters
input_dim = X_train.shape[1]
global_model = create_model(input_dim)
num_rounds = 5
learning_rate = 0.01

global_model = train_fedsgd(global_model, client_data, test_data, num_rounds, learning_rate)
print("Federated training completed.")


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Starting round 1/5...
Client 1 loss: 28.8431
Client 2 loss: 26.2717
Client 3 loss: 26.5010
Global Model Accuracy: 0.0039 (after round 1)
Starting round 2/5...
Client 1 loss: 26.3232
Client 2 loss: 25.4982
Client 3 loss: 24.4029
Global Model Accuracy: 0.0039 (after round 2)
Starting round 3/5...
Client 1 loss: 23.8895
Client 2 loss: 23.1546
Client 3 loss: 21.1018
Global Model Accuracy: 0.0039 (after round 3)
Starting round 4/5...
Client 1 loss: 24.7412
Client 2 loss: 23.5235
Client 3 loss: 20.0852
Global Model Accuracy: 0.0039 (after round 4)
Starting round 5/5...
Client 1 loss: 22.9067
Client 2 loss: 21.9693
Client 3 loss: 21.1666
Global Model Accuracy: 0.0039 (after round 5)
Federated training completed.


In [ ]:
from imblearn.over_sampling import SMOTE
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
file_path = "creditcard.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')

# Handle missing values
data.dropna(inplace=True)

# Separate features and labels
X = data.drop(columns=['Class'])
y = data['Class']

# Address class imbalance using SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Normalize features (excluding Time)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.drop(columns=['Time']))
X_scaled = pd.DataFrame(X_scaled, columns=X.columns[1:])
X_scaled['Time'] = X['Time']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Create federated clients
num_clients = 3
client_data = []
split_X = np.array_split(X_train, num_clients)
split_y = np.array_split(y_train, num_clients)
for i in range(num_clients):
    client_data.append({'X': split_X[i].values, 'y': split_y[i].values})

# Define test data
test_data = {'X': X_test.values, 'y': y_test.values}

# Define model
def create_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Federated SGD training
def train_fedsgd(global_model, client_datasets, test_data, num_rounds, learning_rate):
    global_weights = global_model.get_weights()

    for round_num in range(num_rounds):
        print(f"Starting round {round_num + 1}/{num_rounds}...")
        client_losses = []

        # Local training on each client
        for client_idx, client_data in enumerate(client_datasets):
            # Clone the global model for local training
            local_model = create_model(global_model.input_shape[-1])
            local_model.set_weights(global_weights)

            # Compile the local model
            optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, clipnorm=1.0)
            local_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

            # Train locally for one batch
            history = local_model.fit(client_data['X'], client_data['y'], epochs=1, verbose=0, batch_size=32)

            # Calculate client loss
            local_loss = history.history['loss'][-1]
            client_losses.append(local_loss)
            print(f"Client {client_idx + 1} loss: {local_loss:.4f}")

            # Update global weights (FedSGD: average updates)
            global_weights = [
                global_weight + learning_rate * (local_weight - global_weight)
                for global_weight, local_weight in zip(global_weights, local_model.get_weights())
            ]

        # Set updated global weights
        global_model.set_weights(global_weights)

        # Evaluate global model on test dataset
        global_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        global_loss, global_acc = global_model.evaluate(test_data['X'], test_data['y'], verbose=0)
        print(f"Global Model Accuracy: {global_acc:.4f} (after round {round_num + 1})")

    return global_model

# Training parameters
input_dim = X_train.shape[1]
global_model = create_model(input_dim)
num_rounds = 5
learning_rate = 0.001  # Smaller learning rate for stability

global_model = train_fedsgd(global_model, client_data, test_data, num_rounds, learning_rate)
print("Federated training completed.")


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Starting round 1/5...
Client 1 loss: 675.3394
Client 2 loss: 676.9594
Client 3 loss: 672.4236
Global Model Accuracy: 0.5001 (after round 1)
Starting round 2/5...
Client 1 loss: 671.3728
Client 2 loss: 673.3278
Client 3 loss: 700.3149
Global Model Accuracy: 0.5001 (after round 2)
Starting round 3/5...
Client 1 loss: 663.4209
Client 2 loss: 650.7731
Client 3 loss: 657.4854
Global Model Accuracy: 0.5001 (after round 3)
Starting round 4/5...
Client 1 loss: 665.8766
Client 2 loss: 663.6765
Client 3 loss: 681.2664
Global Model Accuracy: 0.5001 (after round 4)
Starting round 5/5...
Client 1 loss: 667.6034
Client 2 loss: 673.0054
Client 3 loss: 655.2194
Global Model Accuracy: 0.5001 (after round 5)
Federated training completed.


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

# Step 1: Create a synthetic dataset
X, y = make_classification(
    n_samples=2000, n_features=20, n_informative=15, n_redundant=5, random_state=42, weights=[0.95, 0.05]
)

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Normalize the dataset using MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split the data into train/test and further into federated clients
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
client_data = []
num_clients = 3

for i in range(num_clients):
    start = i * len(X_train) // num_clients
    end = (i + 1) * len(X_train) // num_clients
    client_data.append({"X": X_train[start:end], "y": y_train[start:end]})

test_data = {"X": X_test, "y": y_test}

# Step 2: Define the model
def create_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Step 3: Federated Training Function
def train_fedsgd(global_model, client_datasets, test_data, num_rounds, learning_rate):
    global_weights = global_model.get_weights()

    for round_num in range(num_rounds):
        print(f"Starting round {round_num + 1}/{num_rounds}...")
        client_losses = []

        for client_idx, client_data in enumerate(client_datasets):
            # Create and initialize a local model with global weights
            local_model = create_model(global_model.input_shape[-1])
            local_model.set_weights(global_weights)

            # Compile local model
            optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
            local_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

            # Train the local model
            history = local_model.fit(client_data['X'], client_data['y'], epochs=1, verbose=0, batch_size=32)
            client_losses.append(history.history['loss'][-1])

            # Update global weights using averaging
            client_weights = local_model.get_weights()
            global_weights = [(global_w + client_w) / 2 for global_w, client_w in zip(global_weights, client_weights)]

        # Set updated global weights to the global model
        global_model.set_weights(global_weights)

        # Evaluate global model on test data
        global_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        global_loss, global_acc = global_model.evaluate(test_data['X'], test_data['y'], verbose=0)
        print(f"Global Model Accuracy: {global_acc:.4f} (after round {round_num + 1})")

    return global_model

# Step 4: Training the Federated Model
num_rounds = 5
learning_rate = 0.01

global_model = create_model(X_train.shape[1])
global_model = train_fedsgd(global_model, client_data, test_data, num_rounds=num_rounds, learning_rate=learning_rate)

print("Federated training completed.")


Starting round 1/5...
Global Model Accuracy: 0.4947 (after round 1)
Starting round 2/5...
Global Model Accuracy: 0.4934 (after round 2)
Starting round 3/5...
Global Model Accuracy: 0.5608 (after round 3)
Starting round 4/5...
Global Model Accuracy: 0.5966 (after round 4)
Starting round 5/5...
Global Model Accuracy: 0.6257 (after round 5)
Federated training completed.


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and preprocess dataset
file_path = "creditcard.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')

# Drop rows with missing target labels
data = data.dropna(subset=['Class'])

# Separate features and labels
X = data.drop(columns=['Class'])
y = data['Class']

# Normalize features (excluding Time)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.drop(columns=['Time']))
X_scaled = pd.DataFrame(X_scaled, columns=X.columns[1:])
X_scaled['Time'] = X['Time']  # Add Time back unscaled

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Reshape target arrays to 2D
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

# Create federated clients
num_clients = 3
client_data = []
split_X = np.array_split(X_train, num_clients)
split_y = np.array_split(y_train, num_clients)
for i in range(num_clients):
    client_data.append({'X': split_X[i], 'y': split_y[i]})  # Already reshaped to 2D

# Define test data
test_data = {'X': X_test, 'y': y_test}

# Define model
def create_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Federated SGD function
def train_fedsgd(global_model, client_datasets, test_data, num_rounds, learning_rate):
    global_weights = global_model.get_weights()

    for round_num in range(num_rounds):
        print(f"Starting round {round_num + 1}/{num_rounds}...")
        gradients_list = []

        # Collect gradients from clients
        for client_idx, client_data in enumerate(client_datasets):
            with tf.GradientTape() as tape:
                # Clone the global model
                local_model = create_model(global_model.input_shape[-1])
                local_model.set_weights(global_weights)

                # Predict and calculate loss
                predictions = local_model(client_data['X'], training=True)
                loss = tf.keras.losses.binary_crossentropy(client_data['y'], predictions)

            # Compute gradients
            gradients = tape.gradient(loss, local_model.trainable_weights)
            gradients_list.append(gradients)
            print(f"Client {client_idx + 1} loss: {tf.reduce_mean(loss).numpy():.4f}")

        # Average gradients across clients
        averaged_gradients = [tf.reduce_mean([g[layer] for g in gradients_list], axis=0)
                              for layer in range(len(global_weights))]

        # Update global weights using averaged gradients
        global_weights = [
            w - learning_rate * g for w, g in zip(global_weights, averaged_gradients)
        ]
        global_model.set_weights(global_weights)

        # Evaluate global model on test data
        global_loss, global_acc = global_model.evaluate(test_data['X'], test_data['y'], verbose=0)
        print(f"Global Model Accuracy: {global_acc:.4f} (after round {round_num + 1})")

    return global_model

# Initialize global model
input_dim = X_train.shape[1]
global_model = create_model(input_dim)

# Compile global model (required for evaluation)
global_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

# Federated training with SGD
num_rounds = 5
learning_rate = 0.01
global_model = train_fedsgd(global_model, client_data, test_data, num_rounds, learning_rate)
print("Federated training completed.")


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Starting round 1/5...
Client 1 loss: 10199.1562
Client 2 loss: 10183.6768
Client 3 loss: 10154.1348
Global Model Accuracy: 0.9978 (after round 1)
Starting round 2/5...
Client 1 loss: 82588960.0000
Client 2 loss: 70014032.0000
Client 3 loss: 55577120.0000
Global Model Accuracy: 0.9978 (after round 2)
Starting round 3/5...
Client 1 loss: 0.7366
Client 2 loss: 0.5973
Client 3 loss: 0.5176
Global Model Accuracy: 0.9978 (after round 3)
Starting round 4/5...
Client 1 loss: 0.7349
Client 2 loss: 0.5959
Client 3 loss: 0.5165
Global Model Accuracy: 0.9978 (after round 4)
Starting round 5/5...
Client 1 loss: 0.7333
Client 2 loss: 0.5946
Client 3 loss: 0.5153
Global Model Accuracy: 0.9978 (after round 5)
Federated training completed.


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and preprocess dataset
file_path = "creditcard.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')

# Drop rows with missing target labels
data = data.dropna(subset=['Class'])

# Separate features and labels
X = data.drop(columns=['Class'])
y = data['Class']

# Normalize features (excluding Time)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.drop(columns=['Time']))
X_scaled = pd.DataFrame(X_scaled, columns=X.columns[1:])
X_scaled['Time'] = X['Time']  # Add Time back unscaled

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Reshape target arrays to 2D
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

# Create federated clients
num_clients = 3
client_data = []
split_X = np.array_split(X_train, num_clients)
split_y = np.array_split(y_train, num_clients)
for i in range(num_clients):
    client_data.append({'X': split_X[i].values, 'y': split_y[i]})

# Define test data
test_data = {'X': X_test.values, 'y': y_test}

# Define model
def create_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Federated SGD function
def train_fedsgd(global_model, client_datasets, test_data, num_rounds, learning_rate):
    global_weights = global_model.get_weights()

    for round_num in range(num_rounds):
        print(f"Starting round {round_num + 1}/{num_rounds}...")
        gradients_list = []

        # Collect gradients from clients
        for client_idx, client_data in enumerate(client_datasets):
            with tf.GradientTape() as tape:
                # Clone the global model
                local_model = create_model(global_model.input_shape[-1])
                local_model.set_weights(global_weights)

                # Predict and calculate loss
                predictions = local_model(client_data['X'], training=True)
                loss = tf.keras.losses.binary_crossentropy(client_data['y'], predictions)

            # Compute gradients
            gradients = tape.gradient(loss, local_model.trainable_weights)
            gradients_list.append(gradients)
            print(f"Client {client_idx + 1} loss: {tf.reduce_mean(loss).numpy():.4f}")

        # Average gradients across clients
        averaged_gradients = [tf.reduce_mean([g[layer] for g in gradients_list], axis=0)
                              for layer in range(len(global_weights))]

        # Update global weights using averaged gradients
        global_weights = [
            w - learning_rate * g for w, g in zip(global_weights, averaged_gradients)
        ]
        global_model.set_weights(global_weights)

        # Evaluate global model on test data
        global_loss, global_acc = global_model.evaluate(test_data['X'], test_data['y'], verbose=0)
        print(f"Global Model Accuracy: {global_acc:.4f} (after round {round_num + 1})")

    return global_model

# Initialize global model
input_dim = X_train.shape[1]
global_model = create_model(input_dim)

# Compile global model (required for evaluation)
global_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),  # Reduced learning rate
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

# Federated training with SGD
num_rounds = 5
learning_rate = 0.001  # Reduced learning rate
global_model = train_fedsgd(global_model, client_data, test_data, num_rounds, learning_rate)
print("Federated training completed.")


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Starting round 1/5...
Client 1 loss: 6.7456
Client 2 loss: 11.3209
Client 3 loss: 8.4265
Global Model Accuracy: 0.0021 (after round 1)
Starting round 2/5...
Client 1 loss: 5386010099712.0000
Client 2 loss: 5370563002368.0000
Client 3 loss: 5411214721024.0000
Global Model Accuracy: 0.9979 (after round 2)
Starting round 3/5...
Client 1 loss: 75780505600.0000
Client 2 loss: 127139758080.0000
Client 3 loss: 94635548672.0000
Global Model Accuracy: 0.0021 (after round 3)
Starting round 4/5...
Client 1 loss: 705052140550124317179904.0000
Client 2 loss: 703030060346231983439872.0000
Client 3 loss: 708351657781121037369344.0000
Global Model Accuracy: 0.9979 (after round 4)
Starting round 5/5...
Client 1 loss: 199047121633843895186688489553920.0000
Client 2 loss: 333954442122869716583505870192640.0000
Client 3 loss: 248575090953088121242455738679296.0000
Global Model Accuracy: 0.9979 (after round 5)
Federated training completed.


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Sample dataset for binary classification (Replace with your dataset)
# Assume X and y are the feature matrix and target vector respectively
# For this example, we generate a random dataset
X = np.random.rand(1000, 20)  # 1000 samples, 20 features
y = np.random.randint(2, size=1000)  # Binary target variable

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to create the model
def create_model(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Sigmoid activation for binary classification
    return model

# Initialize global model
input_dim = X_train.shape[1]
global_model = create_model(input_dim)

# Compile global model with Adam optimizer
global_model.compile(optimizer=Adam(learning_rate=0.0001),  # Lower learning rate
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

# Function to train federated SGD
def train_fedsgd(global_model, client_data, test_data, num_rounds, learning_rate):
    # Simulate federated learning across clients
    for round_num in range(1, num_rounds + 1):
        print(f"Starting round {round_num}/{num_rounds}...")

        # Train each client and update the global model
        client_losses = []
        for client, (X_train_client, y_train_client) in client_data.items():
            print(f"Training on client {client}...")
            model = create_model(input_dim)  # Create a new model for each client
            model.compile(optimizer=Adam(learning_rate=learning_rate),
                          loss='binary_crossentropy',
                          metrics=['accuracy'])
            model.set_weights(global_model.get_weights())  # Initialize with global model weights
            model.fit(X_train_client, y_train_client, epochs=1, batch_size=32, verbose=0)

            # Evaluate on the test data for the client
            loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
            client_losses.append(loss)
            print(f"Client {client} loss: {loss:.4f}")

            # Update global model with the client's model weights
            global_model.set_weights(model.get_weights())

        # After training on all clients, evaluate global model accuracy
        _, global_accuracy = global_model.evaluate(X_test, y_test, verbose=0)
        print(f"Global Model Accuracy: {global_accuracy:.4f} (after round {round_num})\n")

    return global_model

# Create sample client data (Replace this with actual partitioned data for federated learning)
# Here we create 3 clients by splitting the data randomly
client_data = {
    1: (X_train[:300], y_train[:300]),
    2: (X_train[300:600], y_train[300:600]),
    3: (X_train[600:], y_train[600:])
}

# Number of rounds for federated training
num_rounds = 5

# Train the global model using federated SGD
global_model = train_fedsgd(global_model, client_data, (X_test, y_test), num_rounds, learning_rate=0.0001)

print("Federated training completed.")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Starting round 1/5...
Training on client 1...
Client 1 loss: 0.6838
Training on client 2...
Client 2 loss: 0.6824
Training on client 3...
Client 3 loss: 0.6810
Global Model Accuracy: 0.5750 (after round 1)

Starting round 2/5...
Training on client 1...
Client 1 loss: 0.6799
Training on client 2...
Client 2 loss: 0.6791
Training on client 3...
Client 3 loss: 0.6789
Global Model Accuracy: 0.5750 (after round 2)

Starting round 3/5...
Training on client 1...
Client 1 loss: 0.6789
Training on client 2...
Client 2 loss: 0.6786
Training on client 3...
Client 3 loss: 0.6786
Global Model Accuracy: 0.5750 (after round 3)

Starting round 4/5...
Training on client 1...
Client 1 loss: 0.6790
Training on client 2...
Client 2 loss: 0.6789
Training on client 3...
Client 3 loss: 0.6791
Global Model Accuracy: 0.5950 (after round 4)

Starting round 5/5...
Training on client 1...
Client 1 loss: 0.6793
Training on client 2...
Client 2 loss: 0.6795
Training on client 3...
Client 3 loss: 0.6796
Global Model